---

## 🔹 5. 🧰 **Custom Agents & Logging Extensions**

---

### 📌 **What It Does**

This section focuses on **tracking, extending, and enhancing GenAI/Agentic pipelines** — especially LangChain or LangGraph-based workflows — using custom PythonModel wrappers, prompt tracking, step-level logs, and external logging frameworks.

---

### 🚀 **Common Use in GenAI/Agentic AI**

| Scenario                      | MLflow’s Role                                                 |
| ----------------------------- | ------------------------------------------------------------- |
| LangChain or LangGraph Agents | Wrap full pipelines using `PythonModel`                       |
| Prompt/Chain Versioning       | Save all prompt templates and retriever logic as artifacts    |
| Tool Tracking                 | Use callback-based logging to log each tool call/step         |
| Advanced Feedback Integration | Integrate with tools like Trulens, WandB, or Weights & Biases |

---

### ⚙️ **Key Tools and Techniques**

| Function / Concept                    | Description                                                    |
| ------------------------------------- | -------------------------------------------------------------- |
| `mlflow.pyfunc.PythonModel`           | Base class to wrap agentic pipelines into deployable models    |
| **Prompt Templates as Artifacts**     | Save `.txt` or `.json` prompts for audit/versioning            |
| **Callback-Based Logging**            | Add hooks to log tool usage, retries, inputs/outputs           |
| **External Loggers (Trulens, WandB)** | Add richer feedback tracking: bias, helpfulness, honesty, etc. |

---



### 🔁 Integration with External GenAI Loggers

| Tool                 | Purpose                             | MLflow Integration                        |
| -------------------- | ----------------------------------- | ----------------------------------------- |
| **Trulens**          | Log honesty, bias, coherence        | Export scores and log via `log_metrics()` |
| **Weights & Biases** | Track experiments, graphs, feedback | Use WandB + MLflow side-by-side           |

---

### 🧠 Tips for Agent Tracking

| Task                                  | Recommended Action                                           |
| ------------------------------------- | ------------------------------------------------------------ |
| Store dynamic prompt chains           | Log every `.txt` or `.json` prompt artifact used             |
| Tool invocation analytics             | Add custom callback to log success/failure per tool          |
| External evaluation (Trulens)         | Log with `mlflow.log_metrics()` during/after prediction loop |
| Multi-agent orchestration (LangGraph) | Tag each step/tool output and save chain logs                |

---


In [ ]:
### ✅ Real-Time Example: Wrap LangChain Agent with Logging + Prompt Artifacts


import mlflow
import mlflow.pyfunc
from langchain.agents import initialize_agent, load_tools
from langchain.chat_models import ChatOpenAI

# 1. Define the PythonModel wrapper for your agent
class MyAgentWrapper(mlflow.pyfunc.PythonModel):
    def load_context(self, context):
        tools = load_tools(["serpapi", "llm-math"], llm=ChatOpenAI(model="gpt-4o"))
        self.agent = initialize_agent(tools, ChatOpenAI(), agent="zero-shot-react-description")

    def predict(self, context, inputs):
        query = inputs.iloc[0]  # assuming single-row input
        return self.agent.run(query)

# 2. Save prompt templates or logic as artifacts
with open("prompt_template.txt", "w") as f:
    f.write("Use the toolset wisely and answer: {query}")

# 3. Log everything with MLflow
with mlflow.start_run(run_name="agent-logging"):
    mlflow.pyfunc.log_model(
        artifact_path="agent_model",
        python_model=MyAgentWrapper()
    )
    mlflow.log_artifact("prompt_template.txt")
    mlflow.log_params({"agent_type": "react", "model": "gpt-4o"})
    mlflow.log_metrics({"tools_used": 2, "tool_success_rate": 0.98})




In [ ]:
### 📦 How to Track Steps & Tools (Callback Pattern)

#> For **LangChain**: Use `TracingCallbackHandler` or custom callbacks
#> For **LangGraph**: Add logging inside `@tool` or `@node` decorated functions


from langchain.callbacks.base import BaseCallbackHandler

class MLflowLoggingCallback(BaseCallbackHandler):
    def on_tool_end(self, output, **kwargs):
        mlflow.log_metrics({"last_tool_output_length": len(output)})
